In [1]:
import pandas as pd
import numpy as np

import re
import statsmodels.api as sm

In [2]:
from pathlib import Path
import pandas as pd
CWD = Path.cwd().resolve()
PROJECT_ROOT = CWD if (CWD / "data").exists() else CWD.parent
DATA = PROJECT_ROOT / "data"

In [3]:
rating_map = {
    "AAA": 23, "AA+": 22, "AA": 21, "AA-": 20,
    "A+": 19, "A": 18, "A-": 17,
    "BBB+": 16, "BBB": 15, "BBB-": 14,
    "BB+": 13, "BB": 12, "BB-": 11,
    "B+": 10, "B": 9, "B-": 8,
    "CCC+": 7, "CCC": 6, "CCC-": 5,
    "CC": 4, "C": 3, "SD": 2, "D": 1,
}


def encode_rating(rating):
    """Encode a given rating string into its corresponding numerical value."""
    base_rating = rating.split()[0]  
    base_rating = re.sub(r'^[a-z]+|[a-z]+$|/.*', '', base_rating)
    return rating_map.get(base_rating, 2) 

Convert all metrics into USD

In [4]:
exchange = pd.read_csv(DATA / "raw"/ 'exchange_rate.csv')
exchange = exchange.dropna()
exchange["year"] = pd.to_datetime(exchange["datadate"]).dt.year 
df_grouped = exchange.groupby(["curd", "year"])["exratd_tousd"].mean().reset_index()
exchange_pivot = df_grouped.pivot(index="year", columns="curd", values="exratd_tousd").reset_index()
exchange_pivot = exchange_pivot.set_index("year")

In [5]:
financialgl = pd.read_csv(DATA / "raw"/ 'global_financial.csv')
financialus = pd.read_csv(DATA / "raw"/ 'north_america_financial.csv')
financialus.rename(columns={"curncd": "curcd"})
print(financialgl.shape,financialus.shape)

(912573, 25) (358071, 25)


In [6]:
financial = pd.concat([financialus, financialgl], ignore_index=True, sort=False)
financial['year']  = financial['fyear']
financial = financial.drop_duplicates(subset=['gvkey', 'year'])
financial['ni'] =  financial['ni'].fillna(financial['nicon'])
financial = financial.drop(columns=['nicon'])

In [7]:
cols_to_match = ['at',  'ceq', 'ch', 'dltt', 'ebit','lct', 'lt', 'ni', 'oiadp',  're', 'revt', 'sale','wcap', 'xint']

In [8]:
financial[ "exchange_rate"]  = financial.apply(lambda row: exchange_pivot.loc[row["year"], row["curcd"]] if row["year"] in exchange_pivot.index and row["curcd"] in exchange_pivot.columns else None,axis=1) 
financial[cols_to_match] =financial[cols_to_match].mul(financial["exchange_rate"], axis = 0)

Merge stock price data

In [9]:
security_us = pd.read_csv(DATA / 'na_stock_annual.csv')
security_g = pd.read_csv(DATA / 'global_stock_annual.csv')

In [10]:
financial = financial.merge(security_g, on=['gvkey','year'], how='left')
financial = financial.merge(security_us, on=['gvkey','year'], how='left')

In [11]:
financial['prccm'] = financial[['prccm_x', 'prccm_y']].max(axis=1, skipna=True)

In [12]:
financial['csho_combined'] = financial[['cshoi', 'csho', 'cshoc_y','cshoc_x']].max(axis=1, skipna=True)
financial['mkt_value'] = financial["csho_combined"]*financial["prccm"]

Compute finanical metrics

In [13]:
# Profitability Ratios
financial["ICOV"] = np.where(financial["xint"] <= 0, np.inf, financial["ebit"]/financial["xint"]) 
financial["ROA"] = np.where(financial["at"] > 0, financial["ni"] / financial["at"], np.nan)
financial["OM"] = np.where(financial["revt"] != 0, financial["oiadp"] / financial["revt"], np.nan)

# Leverage Ratios
financial["LDTC"] = np.where((financial["lt"] + financial["ceq"]) != 0, financial["dltt"] / (financial["lt"] + financial["ceq"]), np.nan)
financial["SDTD"] = np.where(financial["lt"] != 0, financial["lct"] / financial["lt"], np.nan)

In [14]:
ratios = ['ROA', 'LDTC', 'ICOV', 'SDTD', 'OM']
for col in ratios:
    max_finite = financial[col][np.isfinite(financial[col])].max()
    min_finite = financial[col][np.isfinite(financial[col])].min()
    financial[col].replace([np.inf], max_finite, inplace=True)
    financial[col].replace([-np.inf], min_finite, inplace=True)

In [15]:
financial['X1'] = financial['wcap'] / abs(financial['at'])
financial['X2'] = financial['re'] / abs(financial['at'])
financial['X3'] = financial['ebit'] /abs(financial['at'])
financial['X4'] = np.where(financial['lt']<=0, np.inf, financial['mkt_value'] /abs(financial['lt']))
financial['X5'] = financial['sale'] / abs(financial['at'])

In [16]:
Xs = ['X1', 'X2','X3','X4','X5']
for col in Xs:
    max_finite = financial[col][np.isfinite(financial[col])].max()
    min_finite = financial[col][np.isfinite(financial[col])].min()
    financial[col].replace([np.inf], max_finite, inplace=True)
    financial[col].replace([-np.inf], min_finite, inplace=True)

In [17]:
# Compute Altman's Z-score
financial['Z_score'] = (1.2 * financial['X1'] + 
                        1.4 * financial['X2'] + 
                        3.3 * financial['X3'] + 
                        0.6 * financial['X4'] + 
                        1.0 * financial['X5'])

In [18]:
financial.to_csv(DATA / "financial.csv", index=False)

Credit data preprocessing

In [19]:
credit = pd.read_csv(DATA / "raw"/ 'ratingdata.csv')

In [20]:
credit['date_column'] = pd.to_datetime(credit['ratingdate'])
years = np.sort(credit['date_column'].dt.year.unique())
credit['year'] = credit['date_column'].dt.year
credit = credit[credit['ratingtypecode'].str.contains('LONG', na=False)]
credit = credit[credit['currentratingsymbol'] != 'NR']
credit = credit[credit['currentratingsymbol'] != 'NR prelim']
credit['numeric_rating'] = credit['currentratingsymbol'].apply(encode_rating)

In [21]:
governs = credit[credit['sectordescription'] == 'Governments']

In [22]:
credit = credit[credit['sectordescription'] == 'Corporates']

In [23]:
credit = credit[credit['priorratingsymbol'].str.strip().str.contains("NR", na=True, case=False) | credit['priorratingsymbol'].str.strip().eq("")]
credit = credit.loc[credit.groupby('entity_id')['year'].idxmin()]

In [24]:
credit = credit.groupby('industrydescription').filter(lambda x: 'Y' in x['unsol'].values)# Filter industries with 'unsol' == 'Y' 

In [25]:
#countryrating
merged = credit.merge(governs, on="countrycode", suffixes=("_credit", "_governs"))
merged = merged[merged["year_governs"] < merged["year_credit"]]
avg_ratings = (
    merged.groupby(["countrycode", "year_credit"])["numeric_rating_governs"]
    .mean()
    .reset_index()
    .rename(columns={"year_credit": "year", "numeric_rating_governs": "countryrating"})
)
credit = credit.merge(avg_ratings, on=["countrycode", "year"], how="left")

In [26]:
credit.to_csv(DATA / "creditdata.csv", index=False)

Merge creidt and financial data

In [27]:
cols_to_match =['ch','wcap',  'ICOV', 'ROA',  'LDTC', 'SDTD',  'OM',  'Z_score', 'xint', 'lt']
merged_df = credit.merge(financial, on='gvkey', suffixes=('_credit', '_financial'), how="left")
merged_df = merged_df[(merged_df['year_financial'] < merged_df['year_credit'] )& (merged_df['year_financial'] >= merged_df['year_credit'] -3)]
avg_values_df = merged_df.groupby('gvkey')[cols_to_match].mean()
final_result = credit.merge(avg_values_df, on='gvkey', how='left')


In [28]:
final_result["unsol"] = final_result["unsol"].map({"N": 1, "Y": 0})

In [29]:
cols_to_match = ['ch', 'wcap', 'ROA',  'ICOV', 'LDTC', 'SDTD',  'OM',  'Z_score']

In [30]:
for rs in cols_to_match:
    Q1 = final_result[rs].quantile(0.25)
    Q3 = final_result[rs].quantile(0.75)
    IQR = Q3 - Q1
    lower_fence = Q1 - 1.5 * IQR
    upper_fence = Q3 + 1.5 * IQR
    lower_extreme_fence = Q1 - 3 * IQR
    upper_extreme_fence = Q3 + 3 * IQR
    final_result[rs] = final_result[rs].clip(lower_extreme_fence, upper_extreme_fence)

In [31]:
final_result.to_csv(DATA / "combined.csv", index=False)

Industry-focused 

In [32]:
credit = pd.read_csv(DATA / "raw"/ 'ratingdata.csv')
credit = credit[~credit['currentratingsymbol'].str.contains("NR", case=False, na=False)]
credit['numeric_rating'] = credit['currentratingsymbol'].apply(encode_rating)
credit['ratingdate'] = pd.to_datetime(credit['ratingdate'], errors='coerce')
credit['year'] = credit['ratingdate'].dt.year
years = np.sort(credit['ratingdate'].dt.year.unique())
governs = credit[credit['sectordescription'] == 'Governments']
credit = credit[credit['ratingtypecode'].str.contains('LONG', na=False)]

In [33]:
credit = credit[credit['sectordescription'] == 'Financial Institutions']
credit = credit[credit['priorratingsymbol'].str.strip().str.contains("NR", na=True, case=False) | credit['priorratingsymbol'].str.strip().eq("")]

In [34]:
credit = credit.groupby('industrydescription').filter(lambda x: 'Y' in x['unsol'].values)
credit = credit.loc[credit.groupby('entity_id')['year'].idxmin()]

In [35]:
merged = credit.merge(governs, on="countrycode", suffixes=("_credit", "_governs"))
merged = merged[merged["year_governs"] < merged["year_credit"]]
avg_ratings = (
    merged.groupby(["countrycode", "year_credit"])["numeric_rating_governs"]
    .mean()
    .reset_index()
    .rename(columns={"year_credit": "year", "numeric_rating_governs": "countryrating"})
)
credit = credit.merge(avg_ratings, on=["countrycode", "year"], how="left")

In [36]:
merged_df = credit.merge(financial, on='gvkey', suffixes=('_credit', '_financial'), how="left")
merged_df = merged_df[(merged_df['year_financial'] < merged_df['year_credit'] )& (merged_df['year_financial'] >= merged_df['year_credit'] -3)]
avg_values_df = merged_df.groupby('gvkey')[cols_to_match].mean()
final_result = credit.merge(avg_values_df, on='gvkey', how='left')

In [37]:
final_result["unsol"] = final_result["unsol"].map({"N": 1, "Y": 0})

In [38]:
for rs in cols_to_match:
    Q1 = final_result[rs].quantile(0.25)
    Q3 = final_result[rs].quantile(0.75)
    IQR = Q3 - Q1
    lower_fence = Q1 - 1.5 * IQR
    upper_fence = Q3 + 1.5 * IQR
    lower_extreme_fence = Q1 - 3 * IQR
    upper_extreme_fence = Q3 + 3 * IQR
    final_result[rs] = final_result[rs].clip(lower_extreme_fence, upper_extreme_fence)

In [39]:
final_result.to_csv(DATA / "combined_industry.csv", index=False)

Empirical calibration

In [40]:
credit = pd.read_csv(DATA / "raw"/ 'ratingdata.csv')
financial = pd.read_csv(DATA / "financial.csv", low_memory=False)
credit['ratingdate'] = pd.to_datetime(credit['ratingdate'], errors='coerce')
credit['year'] = credit['ratingdate'].dt.year
years = np.sort(credit['year'].dropna().unique())[:-1]
credit = credit[credit['sectordescription'] == 'Corporates']
credit['numeric_rating'] = credit['currentratingsymbol'].apply(encode_rating)
credit = credit[credit['ratingtypecode'].str.contains('LONG', na=False)]
credit = credit[~credit['currentratingsymbol'].str.contains("NR", case=False, na=False)]
cols_to_match = ['lt', 'xint']

In [41]:
credit_n = pd.DataFrame(columns = credit.columns)

In [42]:
#if a company has both sol and unsol in one year, mark it as a solicited firm
for year in years:
    year_df = credit[credit['year'] == year]
    mask = year_df.groupby('entity_id')['unsol'].transform(lambda x: 'Y' in x.values and 'N' in x.values)
    year_df.loc[mask, 'unsol'] = 'N'
    credit_n = pd.concat([credit_n, year_df], ignore_index=True)

In [43]:
# Iterate through each year
for year in years:
    year_df = credit_n[credit_n['year'] <= year]
    year_sin = year_df.groupby('countrycode').filter(lambda x: 'Y' in x['unsol'].values)# Filter countries with 'unsol' == 'Y' 
    year_sin = year_sin.groupby("entity_id").tail(1).reset_index(drop=True)
    merged_df = year_sin.merge(financial, on='gvkey', suffixes=('_credit', '_financial'), how="left")
    merged_df = merged_df[(merged_df['year_financial'] <= merged_df['year_credit'] )& (merged_df['year_financial'] > merged_df['year_credit'] -3)]
    avg_values_df = merged_df.groupby('gvkey')[cols_to_match].mean()
    final_result = year_sin.merge(avg_values_df, on='gvkey', how='left')
    final_result["unsol"] = final_result["unsol"].map({"N": 1, "Y": 0})
    final_result.to_csv(DATA / "calibration" / f"{year}_whole.csv", index=False)